In [1]:
from pandas_profiling import ProfileReport
import matplotlib as mpl
import pandas as pd


filename = 'converted/EX0502.csv'
df_data = pd.read_csv(filename).set_index('PERSON')

df_data

,SBP,QUET,AGE,SMK
PERSON,,,,
1,135,2.876,45,0
2,122,3.251,41,0
3,130,3.100,49,0
4,148,3.768,52,0
5,146,2.979,54,1
6,129,2.790,47,1
7,162,3.668,60,1
8,160,3.612,48,1
9,144,2.368,44,1


In [2]:
# initialize matplotlib dpi to get nice crisp hi-res images!
mpl.rcParams['figure.dpi'] = 300

In [3]:
# Systolic Blood Pressure (SBP) [Y],
# Age (AGE) [X1],
# Smoking History (SMK=1 if current or previous smoker, 0 otherwise) [X2],
# and Body Size (QUET) [X3].

Y = 'SBP'
X123 = ['AGE', 'SMK', 'QUET']
X1 = X123[0]
X2 = X123[1]
X3 = X123[2]

In [6]:
import statsmodels.api as sm


x = df_data[X123]
y = df_data[Y]

X = sm.add_constant(x)

model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    SBP   R-squared:                       0.761
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     29.71
Date:                Wed, 18 Aug 2021   Prob (F-statistic):           7.60e-09
Time:                        16:19:04   Log-Likelihood:                -107.35
No. Observations:                  32   AIC:                             222.7
Df Residuals:                      28   BIC:                             228.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         45.1032     10.765      4.190      0.0

/home/thomas/work/nihes/ESP09/venv/lib64/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [9]:
# 1a From the output, provide the overall test, i.e. taken collectively
#   does the entire set of regressors contribute significantly to the 
#   prediction of the response Y. Provide the general construction of 
#   the (F) test statistic, the degrees-of-freedom associated with this
#   statistic, the approximate p-value, and your conclusions in context
#   of the model.
#
import scipy


p_crit = 0.05  # static

dfe = len(X123)  # number of independent variables
dfd = df_data.shape[0] - len(X123)  # number of samples minus number of independent samples
f_crit = scipy.stats.f.ppf(q=1-0.05, dfn=dfe, dfd=dfd)
print('Degrees of freedom nominator: ', dfe)
print('Degrees of freedom denominator: ', dfd)
print('F-crit: ', f_crit)

if results.f_pvalue < p_crit and results.fvalue > f_crit:
    # could also be done with an or since these are tied together
    print('H0 rejected, at least one of the regressors is important')
else:
    print('H0 confirmed, none of the regressors is important')

Degrees of freedom nominator:  3
Degrees of freedom denominator:  29
F-crit:  2.9340298896641714
H0 rejected, at least one of the regressors is important


In [25]:
# 1b The output from the full regression model provides a test for the
#   significance of each variable separately (one test for each 
#   variable). Most commonly the output is in the form of a T-test. 
#   For each of the three regressors: Provide the general construction
#   of the (T) test statistic, the degrees-of-freedom associated with
#   this statistic, the approximate p-value, and your conclusions in
#   context of the model.
#

# General construct:
import numpy as np


n = df_data.shape[0]  # number of samples
beta = 0  # For H0 = 0
calculate_for = 'QUET'  # run on this variable
coeff = results.params.loc[calculate_for]
standard_error = results.bse.loc[calculate_for]

t_val = (coeff - beta) / standard_error

p_val = scipy.stats.t.sf(np.abs(t_val), n-2) * 2  # 2 x area past T

print(f'{calculate_for} T: ', t_val)
print(f'{calculate_for} estimated P: ', p_val)

QUET T:  1.9099927822991845
QUET estimated P:  0.06573606161589875


In [138]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    SBP   R-squared:                       0.761
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     29.71
Date:                Wed, 18 Aug 2021   Prob (F-statistic):           7.60e-09
Time:                        19:08:33   Log-Likelihood:                -107.35
No. Observations:                  32   AIC:                             222.7
Df Residuals:                      28   BIC:                             228.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         45.1032     10.765      4.190      0.000      23.052      67.154
AGE            1.2127      0.324      3.745      0.001       0.549       1.876
SMK            9.9456      2.656      3.744      0.001       4.505      15.386
QUET           8.5924      4.499      1.910      0.066      -0.623      17.808
==============================================================================
Omnibus:                        1.079   Durbin-Watson:                   1.180
Prob(Omnibus):                  0.583   Jarque-Bera (JB):                0.889
Skew:                           0.391   Prob(JB):                        0.641
Kurtosis:                       2.764   Cond. No.                         444.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

54.8622518876492

In [179]:
# 1c Recall that a statistical test can be performed using Extra SS,
#   which is Regression SS (Full Model) –Regression SS (Reduced
#   Model). Let’s first consider such an approach for testing the
#   Age regressor. We need to fit two separate models: one with X1,
#   X2, X3 (full) and a second with only X2, X3 (reduced). 
#   The difference in these two Regression SS is the Extra SS.
#   Recall that the F-test for the X1 test is F*= (Extra SS/ 1)/ MSE(Full residuals).
#   Construct F* (which has degrees of freedom (1, 28)).
#

f_stars = {}
ss_extras = {}
for i in range(0, len(X123)):
    print(f'Leave-out {X123[i]}')
    
    Xs = X123.copy()
    XLo = Xs.pop(i)
    
    x = df_data[Xs]
    y = df_data[Y]

    X = sm.add_constant(x)

    model2 = sm.OLS(y, X)
    results2 = model2.fit()

    extra_ss = results.ess - results2.ess
    print('Extra SS: ', extra_ss)
    
    f_star = (extra_ss / 1) / results.mse_resid
    print(results.mse_total)
    print('F*: ', f_star)
    
    f_stars[i] = f_star
    ss_extras[f'{XLo} | {", ".join(Xs)}'] = extra_ss
    
    # Do the same through statsmodels
    f_test = results.f_test([XLo])
    print(f_test.summary())

Leave-out AGE
Extra SS:  769.4592038652527
207.28931451612902
F*:  14.02529384759863
<F test: F=array([[14.02529385]]), p=0.000828826553627542, df_denom=28, df_num=1>
Leave-out SMK
Extra SS:  769.2334521373259
207.28931451612902
F*:  14.021178964957848
<F test: F=array([[14.02117896]]), p=0.0008300320346821991, df_denom=28, df_num=1>
Leave-out QUET
Extra SS:  200.14146847318898
207.28931451612902
F*:  3.648072428434998
<F test: F=array([[3.64807243]]), p=0.06642677635054356, df_denom=28, df_num=1>


In [180]:
# 1d From the full model, square the T-test statistic associated
#   with Age; thisvalueshould exactly be equal to your calculated
#   F*. Thus the F* and T* (in this case) are equivalent ways of
#   doing partial testing(one variable at a time).

df_test = (results.tvalues ** 2).rename('T*').to_frame().T
series_f_star = pd.Series(f_stars)
series_f_star.index = X123
df_test.loc['F*'] = series_f_star

# assert that they are the same up to 4 decimals
assert df_test[X123].apply(lambda x: round(x[0], 4) == round(x[1], 4)).all()

df_test

,const,AGE,SMK,QUET
T*,17.554828,14.025294,14.021179,3.648072
F*,NaN,14.025294,14.021179,3.648072


In [192]:
# 1e If we were to construct an Extra SS in the same way as we just
#   did for Age, then the list of Extra SS for AGE, SMK, and QUET 
#   respectively would result in TYPE 3 (III) Sums of Squares (or 
#   Partial SS). Such Type III SS are often default information on 
#   statistical software output (such as SAS). Note: the TYPE 3 SS 
#   are *not* (generally) additive to the Model Regression SS (only
#   if done by design with special orthogonal regressors). Provide
#   the Type 3 SS for each regressor, 
#   i.e. SS(X1 | X2, X3), SS(X2| X1, X3), SS (X3 |X1, X2). 
#   Note that these partial Type 3 SS values are the numerator of the
#   F* partial test, i.e. the F*= Type 3 SS/ MSE (full).

# I've already assumed that this question would be asked and generated a dict
# with these numbers when calculating F*. Student - Teacher 1-0.
# Lets convert it to a dataframe and display.
pd.DataFrame.from_dict(ss_extras, orient='index').rename(columns={0: 'SS'})

,SS
"AGE | SMK, QUET",769.459204
"SMK | AGE, QUET",769.233452
"QUET | AGE, SMK",200.141468


In [195]:
# 1f Suppose we would like to test whether the SLR model with SMK is a
#   sufficient reduced model. We then need to determine if we can 
#   reduce the full model (AGE, SMK, QUET) to the reduced model (SMK only).
#   Although it is tempting to simply look at the partial T-tests for AGE
#   and QUET, such an approach does not control for overall testing error
#   (as it is not a joint test, but rather two separate T-tests).  Hence
#   we need to fit two separate models and compute the 
#   Extra SS= SS(Regression Full)-SS(Regression Reduced).
#   In this case, we also write Extra SS=SS(X1,X3 | X2).
#   Recall that the test statistic is now of the form F*=(Extra SS/ 2)/ MSE(Full).
#   Note that the Extra SS is now divided by 2 (not 1 as with partial testing).
#   In general, we divide the Extra SS by the number of parameters set to
#   zero (AGE, QUET here); more generally the divisor is the difference in
#   dimension between the full and reduced models. Thusthis particular F-test
#   has degrees of freedom of (2, 28). Provide the general construction of the
#   conjunctive (F) test statistic, the degrees-of-freedom associated with this
#   statistic, the approximate p-value, and your conclusions in context of the
#   model.Note that virtually any test can be performed in such a way using the
#   conjunctive testing: this is a powerful tool in regression, and the partial
#   tests are simply just a special case.

print('Extra SS=SS(X1,X3 | X2)')
print('F*=(Extra SS/ 2)/ MSE(Full)')
    
Xs = [X2]
XLo = [X1, X3]

x = df_data[Xs]
y = df_data[Y]

X = sm.add_constant(x)

model2 = sm.OLS(y, X)
results2 = model2.fit()

extra_ss = results.ess - results2.ess
print('Extra SS: ', extra_ss)

f_star = (extra_ss / len(XLo)) / results.mse_resid
print(results.mse_total)
print('F*: ', f_star)

# Do the same through statsmodels
f_test = results.f_test([*XLo])
print(f_test.summary())

Extra SS=SS(X1,X3 | X2)
F*=(Extra SS/ 2)/ MSE(Full)
Extra SS:  4496.7275353811165
207.28931451612902
F*:  40.9819810585777
<F test: F=array([[40.98198106]]), p=4.81622896496331e-09, df_denom=28, df_num=2>


In [ ]:
# 1g Type 1 or Sequential SS *are* additive to the model’s Regression SS,
#   so let’s verify this with the Blood Pressure example.  We construct
#   SS1 for Age using the SLR SS Regression for AGE (simple linear
#   regression). We then construct SS Type I for SMK using 
#   SS(SMK | AGE)= Regression SS (AGE, SMK) –Regression SS (AGE). We lastly
#   construct SS Type I for QUET also using Extra SS, i.e. 
#   SS(QUET | AGE, SMK)= Regression SS (AGE, SMK, QUET) –SS (AGE, SMK).
#   Thus to construct the SS Type I, we need to fit several models and
#   construct various SS. Construct the Type I SS for the SBP example.